# SQL Assignment

In [1]:
import pandas as pd
import sqlite3

from IPython.display import display, HTML

In [2]:
# Note that this is not the same db we have used in course videos, please download from this link
# https://drive.google.com/file/d/1O-1-L1DdNxEK6O6nG2jS31MbrMh-OnXM/view?usp=sharing

In [3]:
conn = sqlite3.connect("Db-IMDB-Assignment.db")

#### Overview of all tables

In [4]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
tables = tables["Table_Name"].values.tolist()

In [5]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




## Useful tips:

1. the year column in 'Movie' table, will have few chracters other than numbers which you need to be preprocessed, you need to get a substring of last 4 characters, its better if you convert it as int type, ex: CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)

2. For almost all the TEXT columns we have show, please try to remove trailing spaces, you need to use TRIM() function

3. When you are doing count(coulmn) it won't consider the "NULL" values, you might need to explore other alternatives like Count(*)

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

<h4>To determine whether a year is a leap year, follow these steps:</h4>

<ul>
    <li><b>STEP-1:</b> If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.</li>
    <li><b>STEP-2:</b> If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.</li>
    <li><b>STEP-3:</b> If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.</li>
    <li><b>STEP-4:</b> The year is a leap year (it has 366 days).</li>
    <li><b>STEP-5:</b> The year is not a leap year (it has 365 days).</li>
</ul>

Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.

In [6]:
%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results.head(10))
    print(q1_results.shape)
    assert (q1_results.shape == (232,3))


query1 = """
    SELECT
    TRIM(p.Name) AS Director_Name, 
    TRIM(Movie.title) AS Movie_name, 
    CAST(SUBSTR(TRIM(Movie.year),-4) AS INTEGER) AS release_year
    FROM 
        M_Director md
    JOIN 
        Movie 
    USING(MID)
    JOIN 
        person p
    USING(PID)
    WHERE 
        release_year IN (SELECT  
                        CAST(SUBSTR(TRIM(Movie.year),-4) AS INTEGER) AS movie_year
                        FROM 
                            Movie 
                        WHERE (movie_year % 4 = 0 and movie_year % 100 <> 0) 
                            or (movie_year % 400 = 0) )
        AND Movie.MID IN (SELECT 
                        TRIM(M_Genre.MID)
                        FROM 
                            M_Genre 
                        JOIN 
                            Genre 
                        USING(GID)
                        WHERE Genre.Name LIKE '%Comedy%')  """
grader_1(query1)

      Director_Name                         Movie_name  release_year
0      Milap Zaveri                         Mastizaade          2016
1      Danny Leiner  Harold & Kumar Go to White Castle          2004
2    Anurag Kashyap                 Gangs of Wasseypur          2012
3      Frank Coraci        Around the World in 80 Days          2004
4     Griffin Dunne             The Accidental Husband          2008
5       Anurag Basu                             Barfi!          2012
6   Gurinder Chadha                  Bride & Prejudice          2004
7        Mike Judge    Beavis and Butt-Head Do America          1996
8  Tarun Mansukhani                            Dostana          2008
9      Shakun Batra                      Kapoor & Sons          2016
(232, 3)
Wall time: 489 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [7]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    assert (q2_results.shape == (17,1))


query2 = """
    SELECT 
    Name 
    FROM 
        Person 
    WHERE PID IN (SELECT 
                    TRIM(PID) 
                FROM M_Cast 
                WHERE MID == (SELECT MID 
                            FROM Movie 
                            WHERE TRIM(title) = 'Anand')) """
grader_2(query2)

                Name
0   Amitabh Bachchan
1      Rajesh Khanna
2      Sumita Sanyal
3         Ramesh Deo
4          Seema Deo
5     Asit Kumar Sen
6         Dev Kishan
7       Atam Prakash
8      Lalita Kumari
9             Savita
Wall time: 271 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [8]:
%%time

def grader_3a(query_less_1970, query_more_1990):
    q3_a = pd.read_sql_query(query_less_1970,conn)
    print(q3_a.shape)
    q3_b = pd.read_sql_query(query_more_1990,conn)
    print(q3_b.shape)
    return (q3_a.shape == (4942,1)) and (q3_b.shape == (62570,1))

query_less_1970 =""" 
SELECT 
p.PID 
FROM Person p 
INNER JOIN 
(
    SELECT TRIM(mc.PID) PD, mc.MID 
    FROM M_cast mc 
    WHERE mc.MID IN
    (
    SELECT mv.MID FROM Movie mv WHERE CAST(SUBSTR(mv.year,-4) AS INTEGER) < 1970
    )
) r1 
    ON r1.PD=p.PID 
"""

query_more_1990 =""" 
SELECT
p.PID 
FROM Person p 
INNER JOIN 
(
    SELECT TRIM(mc.PID) PD, mc.MID FROM M_cast mc 
    WHERE mc.MID IN 
    (
    SELECT mv.MID FROM Movie mv WHERE CAST(SUBSTR(mv.year,-4) AS INTEGER)>1990
    )
) r1 
on r1.PD=p.PID """
print(grader_3a(query_less_1970, query_more_1990))

# using the above two queries, you can find the answer to the given question 

(4942, 1)
(62570, 1)
True
Wall time: 914 ms


In [9]:
%%time
def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    assert (q3_results.shape == (300,1))

query3 = """ 
    SELECT Name
    FROM Person
    WHERE PID IN (Select p.PID from Person p 
                INNER JOIN 
                (
                    SELECT TRIM(mc.PID) PD, mc.MID from M_cast mc 
                    WHERE mc.MID IN 
                    (
                    SELECT mv.MID from Movie mv WHERE CAST(SUBSTR(mv.year,-4) AS INTEGER)<1970
                    )
                    ) r1 
                    ON r1.PD=p.PID) 
    AND PID IN (Select p.PID from Person p 
                INNER JOIN 
                (
                    SELECT TRIM(mc.PID) PD, mc.MID FROM M_cast mc 
                    WHERE mc.MID 
                    IN 
                (
                    SELECT mv.MID FROM Movie mv WHERE CAST(SUBSTR(mv.year,-4) AS INTEGER)>1990
                )
                ) r1 
                    ON r1.PD=p.PID) """
grader_3(query3)

                Name
0       Rishi Kapoor
1   Amitabh Bachchan
2             Asrani
3       Zohra Sehgal
4    Parikshat Sahni
5      Rakesh Sharma
6        Sanjay Dutt
7          Ric Young
8              Yusuf
9     Suhasini Mulay
Wall time: 876 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [10]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10)) 
    return (query_4a.shape == (1462,2))

query_4a =""" 
    SELECT 
    PID AS Director_ID, 
    COUNT(MID) AS Movie_Count 
    FROM M_Director 
    GROUP BY PID """
print(grader_4a(query_4a))

# using the above query, you can write the answer to the given question

  Director_ID  Movie_Count
0   nm0000180            1
1   nm0000187            1
2   nm0000229            1
3   nm0000269            1
4   nm0000386            1
5   nm0000487            2
6   nm0000965            1
7   nm0001060            1
8   nm0001162            1
9   nm0001241            1
True
Wall time: 14.8 ms


In [11]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    assert (q4_results.shape == (58,2))

query4 = """ SELECT 
                p.Name, 
                r.Movie_count 
            FROM (SELECT PID, COUNT(*) as Movie_count
                FROM M_Director
                GROUP BY PID 
                HAVING COUNT(*) >= 10
                ) r
            JOIN Person p
            USING (PID)
            ORDER BY r.Movie_count DESC
            """
grader_4(query4)

                    Name  Movie_count
0           David Dhawan           39
1           Mahesh Bhatt           35
2        Ram Gopal Varma           30
3           Priyadarshan           30
4           Vikram Bhatt           29
5   Hrishikesh Mukherjee           27
6            Yash Chopra           21
7         Shakti Samanta           19
8        Basu Chatterjee           19
9           Subhash Ghai           18
Wall time: 62.5 ms


In [12]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    assert (q4_results.shape == (58,2))

query4 = """ SELECT p.Name, r.Movie_Count 
              FROM Person p 
              INNER JOIN (
                          SELECT PID AS Director_ID, COUNT(MID) AS Movie_Count 
                          FROM M_Director 
                          GROUP BY PID
                          ) r 
              ON p.PID = r.Director_ID
              WHERE r.Movie_Count >= 10
              ORDER BY r.Movie_Count DESC  """
grader_4(query4)

                    Name  Movie_Count
0           David Dhawan           39
1           Mahesh Bhatt           35
2        Ram Gopal Varma           30
3           Priyadarshan           30
4           Vikram Bhatt           29
5   Hrishikesh Mukherjee           27
6            Yash Chopra           21
7         Shakti Samanta           19
8        Basu Chatterjee           19
9           Subhash Ghai           18
Wall time: 45.9 ms


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [13]:
%%time

# note that you don't need TRIM for person table

def grader_5aa(query_5aa):
    query_5aa = pd.read_sql_query(query_5aa,conn)
    print(query_5aa.head(10)) 
    return (query_5aa.shape == (8846,3))

query_5aa ="""
    SELECT 
    r.MID, p.Gender, COUNT(*) HC
    FROM Person p
    INNER JOIN (
                SELECT TRIM(PID) pid, TRIM(MID) mid FROM M_Cast 
                ) r
    ON r.pid = p.PID
    GROUP BY TRIM(r.mid), p.Gender
                
              """

print(grader_5aa(query_5aa))

def grader_5ab(query_5ab):
    query_5ab = pd.read_sql_query(query_5ab,conn)
    print(query_5ab.head(10)) 
    return (query_5ab.shape == (3469, 3))

query_5ab =""" 
    SELECT r.MID, p.Gender, COUNT(*) HC
    FROM Person p
    INNER JOIN (
                    SELECT TRIM(PID) pid, TRIM(MID) mid FROM M_Cast 
                                        ) r
    ON r.pid = p.PID
    WHERE p.Gender = 'Male'
    GROUP BY TRIM(r.mid), p.Gender
                
                 """

print(grader_5ab(query_5ab))


# using the above queries, you can write the answer to the given question

         mid  Gender  HC
0  tt0021594    None   1
1  tt0021594  Female   3
2  tt0021594    Male   5
3  tt0026274    None   2
4  tt0026274  Female  11
5  tt0026274    Male   9
6  tt0027256    None   2
7  tt0027256  Female   5
8  tt0027256    Male   8
9  tt0028217  Female   3
True
         mid Gender  HC
0  tt0021594   Male   5
1  tt0026274   Male   9
2  tt0027256   Male   8
3  tt0028217   Male   7
4  tt0031580   Male  27
5  tt0033616   Male  46
6  tt0036077   Male  11
7  tt0038491   Male   7
8  tt0039654   Male   6
9  tt0040067   Male  10
True
Wall time: 1.28 s


In [14]:
%%time
def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    print(q5a_results.shape)
    assert (q5a_results.shape == (4,2))

query5a = """
        SELECT 
        CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) Year, 
        COUNT(d.mid) Female_only
        FROM Movie m
        INNER JOIN (SELECT *
                    FROM (SELECT (r.MID) mid , (p.Gender) gender, COUNT(*) HC
                        FROM Person p
                        INNER JOIN (
                            SELECT TRIM(PID) pid, TRIM(MID) mid FROM M_Cast 
                                        ) r
                        ON r.pid = p.PID
                        WHERE p.Gender = 'Female'
                        GROUP BY TRIM(r.mid), p.Gender) a
                    INNER JOIN (SELECT (r.MID) mid, COUNT(*) HC
                        FROM Person p
                        INNER JOIN (
                            SELECT TRIM(PID) pid, TRIM(MID) mid FROM M_Cast 
                                        ) r
                        ON r.pid = p.PID
                        GROUP BY TRIM(r.mid)) b
                    USING (mid)
                    WHERE a.HC = b.HC) d
        ON m.MID = d.mid
        GROUP BY CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) """
grader_5a(query5a)

   Year  Female_only
0  1939            1
1  1999            1
2  2000            1
3  2018            1
(4, 2)
Wall time: 1.02 s


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [15]:
%%time
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    assert (q5b_results.shape == (4,3))

query5b = """
        SELECT 
        Year, 
        (Female_only*100.0)/total_count Percentage_female_movies, 
        total_count 
        FROM (SELECT CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) Year, COUNT(d.mid) Female_only
                FROM Movie m
        INNER JOIN (SELECT *
                    FROM (SELECT (r.MID) mid , (p.Gender) gender, COUNT(*) HC
                        FROM Person p
                        INNER JOIN (
                                SELECT TRIM(PID) pid, TRIM(MID) mid FROM M_Cast 
                                        ) r
                        ON r.pid = p.PID
                        WHERE p.Gender = 'Female'
                        GROUP BY TRIM(r.mid), p.Gender) a
                    INNER JOIN (SELECT (r.MID) mid, COUNT(*) HC
                                FROM Person p
                                INNER JOIN (
                                    SELECT TRIM(PID) pid, TRIM(MID) mid FROM M_Cast 
                                        ) r
                                ON r.pid = p.PID
                                GROUP BY TRIM(r.mid)) b
                    USING (mid)
                    WHERE a.HC = b.HC) d
            ON m.MID = d.mid
            GROUP BY CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)) y
            INNER JOIN (SELECT CAST(SUBSTR(TRIM(year),-4) AS INTEGER) Year1, 
                        COUNT(*) total_count
                        FROM Movie
                        GROUP BY Year1) x
            ON x.Year1 = y.Year
            """
grader_5b(query5b)

   Year  Percentage_female_movies  total_count
0  1939                 50.000000            2
1  1999                  1.515152           66
2  2000                  1.562500           64
3  2018                  0.961538          104
Wall time: 1.09 s


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [16]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    assert (q6_results.shape == (3473, 2))

query6 = """
        SELECT 
        m.title Title, 
        r.cast_count 
        FROM Movie m
        LEFT JOIN (SELECT MID, COUNT(PID) cast_count
        FROM M_Cast
        GROUP BY MID) r
        ON m.MID = r.MID
        ORDER BY cast_count DESC
        """
grader_6(query6)

                        Title  cast_count
0               Ocean's Eight         238
1                    Apaharan         233
2                        Gold         215
3             My Name Is Khan         213
4  Captain America: Civil War         191
5                    Geostorm         170
6                     Striker         165
7                        2012         154
8                      Pixels         144
9       Yamla Pagla Deewana 2         140
Wall time: 186 ms


### Q7 --- A decade is a sequence of 10 consecutive years. 
### For example, say in your database you have movie information starting from 1931. 
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on. 
### Find the decade D with the largest number of films and the total number of films in D

In [17]:
%%time
def grader_7a(q7a):
    q7a_results  = pd.read_sql_query(q7a,conn)
    print(q7a_results.head(10))
    assert (q7a_results.shape == (78, 2))

query7a = """ SELECT DISTINCT CAST(SUBSTR(TRIM(year),-4) AS INTEGER) year, COUNT(title) Count
              FROM Movie 
              GROUP BY CAST(SUBSTR(TRIM(year),-4) AS INTEGER)"""
grader_7a(query7a)

# using the above query, you can write the answer to the given question

   year  Count
0  1931      1
1  1936      3
2  1939      2
3  1941      1
4  1943      1
5  1946      2
6  1947      2
7  1948      3
8  1949      3
9  1950      2
Wall time: 20.5 ms


In [18]:
%%time
def grader_7b(q7b):
    q7b_results  = pd.read_sql_query(q7b,conn)
    print(q7b_results.head(10))
    assert (q7b_results.shape == (713, 4))

query7b = """ 
          SELECT *
          FROM (SELECT CAST(SUBSTR(TRIM(year),-4) AS INTEGER) year, COUNT(title) Count
                FROM Movie  
                GROUP BY CAST(SUBSTR(TRIM(year),-4) AS INTEGER)) r,(
                                                                    SELECT DISTINCT CAST(SUBSTR(TRIM(year),-4) AS INTEGER) year2, COUNT(title) Count2
                                                                    FROM Movie 
                                                                    GROUP BY CAST(SUBSTR(TRIM(year),-4) AS INTEGER)
                                                                    ) b
          WHERE b.year2 <= r.year+9 AND b.year2 >= r.year
          ORDER BY r.year
     
          """
grader_7b(query7b)
# if you see the below results the first movie year is less than 2nd movie year and 
# 2nd movie year is less or equal to the first movie year+9

# using the above query, you can write the answer to the given question

   year  Count  year2  Count2
0  1931      1   1931       1
1  1931      1   1936       3
2  1931      1   1939       2
3  1936      3   1936       3
4  1936      3   1939       2
5  1936      3   1941       1
6  1936      3   1943       1
7  1939      2   1939       2
8  1939      2   1941       1
9  1939      2   1943       1
Wall time: 46.8 ms


In [25]:
%%time
def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results.head(10))
    assert (q7_results.shape == (1, 2))

query7 = """ 
            SELECT  year as Decade, MAX(total) as Max_number_of_films
            FROM (SELECT SUM(Count2) total, year
                  FROM (SELECT *
                        FROM (SELECT DISTINCT CAST(SUBSTR(TRIM(year),-4) AS INTEGER) year, COUNT(title) Count
                              FROM Movie  
                              GROUP BY CAST(SUBSTR(TRIM(year),-4) AS INTEGER)) r,(
                                                                    SELECT DISTINCT CAST(SUBSTR(TRIM(year),-4) AS INTEGER) year2, COUNT(title) Count2
                                                                    FROM Movie 
                                                                    GROUP BY CAST(SUBSTR(TRIM(year),-4) AS INTEGER)
                                                                    ) b
            WHERE b.year2 <= r.year+9 AND b.year2 >= r.year
            ORDER BY r.year) 
            GROUP BY year)"""
grader_7(query7)
# if you check the output we are printinng all the year in that decade, its fine you can print 2008 or 2008-2017

   Decade  Max_number_of_films
0    2008                 1203
Wall time: 39 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [20]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

query8a = """ 
        SELECT 
        TRIM(c.PID) actor, 
        TRIM(r.PID) director, 
        COUNT(MID) movies
        FROM M_Director r 
        INNER JOIN M_Cast c 
        USING(MID) 
        GROUP BY actor, director
              ORDER BY movies DESC           
              """
grader_8a(query8a)

# using the above query, you can write the answer to the given question

       actor   director  movies
0  nm0456094  nm0223522      23
1  nm0007106  nm0223522      20
2  nm0434318  nm0223522      20
3  nm0318622  nm0080315      19
4  nm0332871  nm0223522      17
5  nm0712546  nm0698184      16
6  nm2147526  nm0698184      15
7  nm0442479  nm0223522      14
8  nm0451272  nm0080315      14
9  nm0451600  nm0223522      14
Wall time: 1.12 s


In [21]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

query8 = """
        SELECT 
        p.Name Name, 
        r.movies1 count
        FROM (SELECT TRIM(c.PID) actor1, 
            TRIM(r.PID) director1, COUNT(MID) movies1
            FROM M_Director r 
            INNER JOIN M_Cast c 
            USING(MID) 
            GROUP BY actor1, director1) r
            INNER JOIN Person p
            ON r.actor1 = p.PID
            WHERE r.actor1 NOT IN (SELECT DISTINCT(a.actor1)
                                FROM (SELECT TRIM(c.PID) actor1, TRIM(r.PID) director1, COUNT(MID) movies1
                                      FROM M_Director r 
                                      INNER JOIN M_Cast c 
                                      USING(MID) 
                                      GROUP BY actor1, director1) a,
                                      (SELECT TRIM(c.PID) actor2, TRIM(r.PID) director2, COUNT(MID) movies2
                                        FROM M_Director r 
                                        INNER JOIN M_Cast c 
                                        USING(MID) 
                                        GROUP BY actor2, director2) b
            WHERE a.director1 =  (SELECT PID 
                                FROM Person 
                                WHERE TRIM(Name) = "Yash Chopra") AND a.actor1 = b.actor2 AND a.movies1 < b.movies2
                                ORDER BY b.movies2 DESC)
            AND r.director1 = (SELECT PID 
                                FROM Person 
                                WHERE TRIM(Name) = "Yash Chopra")
            ORDER BY r.movies1 DESC
             
            """
grader_8(query8)

                Name  count
0        Jagdish Raj     11
1   Manmohan Krishna     10
2           Iftekhar      9
3      Shashi Kapoor      7
4      Rakhee Gulzar      5
5     Waheeda Rehman      5
6           Ravikant      4
7     Achala Sachdev      4
8        Neetu Singh      4
9      Leela Chitnis      3
(245, 2)
Wall time: 3.27 s


In [22]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

query8 = """ 
            SELECT 
            p.Name Name, 
            r.movies1 count
            FROM (SELECT TRIM(c.PID) actor1, TRIM(r.PID) director1, COUNT(MID) movies1
                  FROM M_Director r 
                  INNER JOIN M_Cast c 
                  USING(MID) 
                  GROUP BY actor1, director1) r
            INNER JOIN Person p
            ON r.actor1 = p.PID
            WHERE r.actor1 NOT IN (SELECT DISTINCT(a.actor1)
                                    FROM (SELECT TRIM(c.PID) actor1, TRIM(r.PID) director1, COUNT(MID) movies1
                                    FROM M_Director r 
                                    INNER JOIN M_Cast c 
                                    USING(MID) 
                                    GROUP BY actor1, director1) a,
                                          (SELECT TRIM(c.PID) actor2, TRIM(r.PID) director2, COUNT(MID) movies2
                                            FROM M_Director r 
                                            INNER JOIN M_Cast c 
                                            USING(MID) 
                                            GROUP BY actor2, director2) b
            WHERE a.director1 =  (SELECT PID 
                                FROM Person 
                                WHERE TRIM(Name) = "Yash Chopra") AND a.actor1 = b.actor2 AND a.movies1 < b.movies2
                                ORDER BY b.movies2 DESC)
            AND r.director1 = (SELECT PID 
                                FROM Person 
                                WHERE TRIM(Name) = "Yash Chopra")
            ORDER BY r.movies1 DESC
             
            
             """
grader_8(query8)

                Name  count
0        Jagdish Raj     11
1   Manmohan Krishna     10
2           Iftekhar      9
3      Shashi Kapoor      7
4      Rakhee Gulzar      5
5     Waheeda Rehman      5
6           Ravikant      4
7     Achala Sachdev      4
8        Neetu Singh      4
9      Leela Chitnis      3
(245, 2)
Wall time: 3.48 s


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [23]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    assert (q9a_results.shape == (2382, 1))

query9a = """ 
        SELECT DISTINCT(TRIM(PID)) as Pearson_id
        FROM M_Cast
        WHERE TRIM(MID) IN (
                        SELECT TRIM(MID) FROM M_Cast 
                        WHERE TRIM(PID) = (SELECT PID FROM Person 
                                            WHERE TRIM(Name) = 'Shah Rukh Khan')) AND TRIM(PID) <> (SELECT PID FROM Person WHERE TRIM(Name) = 'Shah Rukh Khan') 
                      """
grader_9a(query9a)
# using the above query, you can write the answer to the given question

# selecting actors who acted with srk (S1)
# selecting all movies where S1 actors acted, this forms S2 movies list
# selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors
# removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors

  Pearson_id
0  nm0004418
1  nm1995953
2  nm2778261
3  nm0631373
4  nm0241935
5  nm0792116
6  nm1300111
7  nm0196375
8  nm1464837
9  nm2868019
(2382, 1)
Wall time: 144 ms


In [24]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

query9 = """ SELECT Name
            FROM Person 
            WHERE TRIM(PID) IN (SELECT DISTINCT (TRIM(PID))
                                FROM (SELECT MID, PID
                                    FROM M_Cast
                                    WHERE TRIM(MID) IN (SELECT MID 
                                                        FROM M_CAST 
                                                        WHERE TRIM(PID) IN (SELECT TRIM(PID)
                                                                            FROM M_Cast
                                                                            WHERE TRIM(MID) IN (SELECT MID 
                                                                            FROM M_Cast WHERE TRIM(PID) = (SELECT PID 
                                                                                                            FROM Person WHERE TRIM(Name) = 'Shah Rukh Khan')))))
            WHERE TRIM(PID) NOT IN (SELECT TRIM(PID)
                                    FROM M_Cast
            WHERE TRIM(MID) IN (SELECT MID 
                                FROM M_Cast 
                                WHERE TRIM(PID) = (SELECT PID 
                                                    FROM Person WHERE TRIM(Name) = 'Shah Rukh Khan'))))           """
grader_9(query9)

                     Name
0            Freida Pinto
1             Rohan Chand
2            Damian Young
3         Waris Ahluwalia
4   Caroline Christl Long
5           Rajeev Pahuja
6       Michelle Santiago
7         Alicia Vikander
8            Dominic West
9          Walton Goggins
(25698, 1)
Wall time: 968 ms
